<a href="https://colab.research.google.com/github/princealexanderm/ICT_DAS_Prince_Alex/blob/main/ICT_Project_Australian_Weather_Data_Best_Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Australia Weather Prediction:Rain Tomorrow

**Best model is :df_reg_random_cap_log_encoded_scaled**

w

Missing value handling
 - Numerical: LinearRegression imputation
 - Categorical:Frequency based random

Outlier:
 - Capping
 - log transformation (extreme but genuine)

Encoding
 - OneHotEncoding

Standardization
 - StandardScaler


Data: https://www.kaggle.com/datasets/arunavakrchakraborty/australia-weather-data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/Weather Training Data.csv')
df.head(10)

,row ID,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Row0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0
1,Row1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0
2,Row2,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0
3,Row3,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,0
4,Row4,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,0
5,Row5,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,0
6,Row6,Albury,13.4,30.4,0.0,NaN,NaN,N,30.0,SSE,...,48.0,22.0,1011.8,1008.7,NaN,NaN,20.4,28.8,No,1
7,Row7,Albury,15.9,21.7,2.2,NaN,NaN,NNE,31.0,NE,...,89.0,91.0,1010.5,1004.2,8.0,8.0,15.9,17.0,Yes,1
8,Row8,Albury,12.6,21.0,3.6,NaN,NaN,SW,44.0,W,...,65.0,43.0,1001.2,1001.8,NaN,7.0,15.8,19.8,Yes,0
9,Row9,Albury,9.8,27.7,NaN,NaN,NaN,WNW,50.0,NaN,...,50.0,28.0,1013.4,1010.3,0.0,NaN,17.3,26.2,NaN,0


In [ ]:
df=df.drop(['row ID'],axis=1)

In [ ]:
print(df.isna().sum())

Location             0
MinTemp            443
MaxTemp            230
Rainfall           979
Evaporation      42531
Sunshine         47317
WindGustDir       6521
WindGustSpeed     6480
WindDir9am        7006
WindDir3pm        2648
WindSpeed9am       935
WindSpeed3pm      1835
Humidity9am       1233
Humidity3pm       2506
Pressure9am       9748
Pressure3pm       9736
Cloud9am         37572
Cloud3pm         40002
Temp9am            614
Temp3pm           1904
RainToday          979
RainTomorrow         0
dtype: int64


In [ ]:
df_reg_random=df.copy()

## Missing Value Handling

**Fill missing values in columns_numerical based on RainTomorrow uisng LinearRegression using loop**

In [ ]:
columns_numerical=['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm']

In [ ]:
from sklearn.linear_model import LinearRegression
for column in columns_numerical:
  # Create a copy of the dataframe
  temp_df = df_reg_random.copy()

  # Separate rows with and without missing values for the target column
  df_not_missing = temp_df.dropna(subset=[column])
  df_missing = temp_df[temp_df[column].isnull()]

  # Check if there are any missing values in the current column
  if df_missing.empty:
    continue

  # Define X (features) and y (target) for training
  X_train = df_not_missing[['RainTomorrow']]  # Assuming 'RainTomorrow' is a feature correlated with missing values
  y_train = df_not_missing[column]

  # Create and train a linear regression model
  regressor = LinearRegression()
  regressor.fit(X_train, y_train)

  # Predict missing values
  X_missing = df_missing[['RainTomorrow']]
  df_missing[column] = regressor.predict(X_missing)

  # Replace missing values in the original dataframe
  df_reg_random.loc[df_reg_random[column].isnull(), column] = df_missing[column]

<ipython-input-9-226995fb0a06>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing[column] = regressor.predict(X_missing)
<ipython-input-9-226995fb0a06>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing[column] = regressor.predict(X_missing)
<ipython-input-9-226995fb0a06>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

**Fill missing values in categorical column using random frequency metho**

In [ ]:
categorical_columns = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [ ]:
for column in categorical_columns:
  # Calculate the frequency distribution of non-missing values
  value_counts = df_reg_random[column].value_counts(normalize=True)

  # Sample from the distribution to fill missing values
  df_reg_random[column].fillna(pd.Series(np.random.choice(value_counts.index, size=len(df_reg_random[column]), p=value_counts.values)), inplace=True)

<ipython-input-11-22fd8736cdc4>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_reg_random[column].fillna(pd.Series(np.random.choice(value_counts.index, size=len(df_reg_random[column]), p=value_counts.values)), inplace=True)
<ipython-input-11-22fd8736cdc4>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [ ]:
print(df_reg_random.isna().sum())

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64


## Outlier Handling

In [ ]:
df_reg_random_cap_log = df_reg_random.copy()

columns_to_cap = ['MinTemp', 'MaxTemp', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']
columns_to_log_transform = ['Rainfall', 'Evaporation', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']


for column in columns_to_cap:
    Q1 = df_reg_random_cap_log[column].quantile(0.25)
    Q3 = df_reg_random_cap_log[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_reg_random_cap_log[column] = np.where(df_reg_random_cap_log[column] > upper_bound, upper_bound,
                                         np.where(df_reg_random_cap_log[column] < lower_bound, lower_bound,
                                                  df_reg_random_cap_log[column]))

for column in columns_to_log_transform:
    Q1 = df_reg_random_cap_log[column].quantile(0.25)
    Q3 = df_reg_random_cap_log[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_reg_random_cap_log.loc[df_reg_random_cap_log[column] > upper_bound, column] = np.log1p(df_reg_random_cap_log.loc[df_reg_random_cap_log[column] > upper_bound, column]+1)
    df_reg_random_cap_log.loc[df_reg_random_cap_log[column] < lower_bound, column] = np.log1p(df_reg_random_cap_log.loc[df_reg_random_cap_log[column] < lower_bound, column]+1)



## Encoding

In [ ]:
categorical=['Location', 'WindGustDir','WindDir9am', 'WindDir3pm','RainToday']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

df_reg_random_cap_log_encoded = df_reg_random_cap_log.copy()

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform the encoder on the categorical columns
encoded_features = encoder.fit_transform(df_reg_random_cap_log_encoded[categorical])

# Create a new DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical))

# Concatenate the encoded DataFrame with the original DataFrame
df_reg_random_cap_log_encoded = pd.concat([df_reg_random_cap_log_encoded, encoded_df], axis=1)

# Drop the original categorical columns
df_reg_random_cap_log_encoded = df_reg_random_cap_log_encoded.drop(columns=categorical)

df_reg_random_cap_log_encoded.head(2)


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
0,13.4,22.9,0.6,5.716563,8.537287,44.0,20.0,24.0,71.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,7.4,25.1,0.0,5.716563,8.537287,44.0,4.0,22.0,44.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


## Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the scaler on the DataFrame, excluding the target variable (assuming 'RainTomorrow' is the target)
if 'RainTomorrow' in df_reg_random_cap_log_encoded.columns:
  X = df_reg_random_cap_log_encoded.drop('RainTomorrow', axis=1)
  y = df_reg_random_cap_log_encoded['RainTomorrow']
  X_scaled = scaler.fit_transform(X)
  df_reg_random_cap_log_encoded_scaled = pd.DataFrame(X_scaled, columns=X.columns)
  df_reg_random_cap_log_encoded_scaled['RainTomorrow'] = y
else:
  X = df_reg_random_cap_log_encoded.copy()
  X_scaled = scaler.fit_transform(X)
  df_reg_random_cap_log_encoded_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Now df_reg_random_cap_log_encoded_scaled contains the standardized data.
df_reg_random_cap_log_encoded_scaled.head(2)


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes,RainTomorrow
0,0.191844,-0.045444,0.040982,0.665737,0.305777,0.530685,0.787432,0.745039,0.110185,-1.431757,...,-0.287835,-0.265414,-0.24736,-0.266864,-0.276482,3.877224,-0.271608,0.537339,-0.537339,0
1,-0.749230,0.264900,-0.580685,0.665737,0.305777,0.530685,-1.155281,0.499455,-1.324554,-1.285938,...,-0.287835,-0.265414,-0.24736,-0.266864,-0.276482,-0.257916,3.681780,0.537339,-0.537339,0


##Model Selection

In [ ]:
df=df_reg_random_cap_log_encoded_scaled

In [ ]:
X=df.drop('RainTomorrow',axis=1)
y=df['RainTomorrow']

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X_train,y_train)
y_pred = logit_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score,mean_absolute_error

print('Accuracy is', accuracy_score(y_test, y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))
print('R2 score',r2_score(y_test,y_pred))

Accuracy is 0.8868066720257235
MAE 0.11319332797427653
R2 score 0.35327604906411336


In [ ]:
y_pred

array([0, 1, 0, ..., 0, 0, 0])

**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model=KNeighborsClassifier()
knn_model.fit(X_train,y_train)

KNeighborsClassifier()

In [ ]:
y_pred=knn_model.predict(X_test)

print('Accuracy is', accuracy_score(y_test, y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))
print('R2 score',r2_score(y_test,y_pred))

Accuracy is 0.827371382636656
MAE 0.17262861736334406
R2 score 0.013695741049397858


**SVC**

In [ ]:
from sklearn.svm import SVC

svc_model=SVC()
svc_model.fit(X_train,y_train)

SVC()

In [ ]:
y_pred=svc_model.predict(X_test)

print('Accuracy is', accuracy_score(y_test, y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))
print('R2 score',r2_score(y_test,y_pred))

Accuracy is 0.9241860932475884
MAE 0.07581390675241158
R2 score 0.5668413484410773


**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model=DecisionTreeClassifier()
dt_model.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred=dt_model.predict(X_test)

print('Accuracy is', accuracy_score(y_test, y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))
print('R2 score',r2_score(y_test,y_pred))

Accuracy is 0.9065514469453376
MAE 0.09344855305466238
R2 score 0.4660867515575903


**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model=RandomForestClassifier()
rf_model.fit(X_train,y_train)

In [ ]:
y_pred=rf_model.predict(X_test)

print('Accuracy is', accuracy_score(y_test, y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))
print('R2 score',r2_score(y_test,y_pred))

Accuracy is 0.9365956591639871
MAE 0.06340434083601286
R2 score 0.6377427314331607


**Random Forest Generated an accuracy of 93.6%**

---